# Web Scraping

Building a function to return the text and other relevant information from blog posts and articles in order to build the training set for the classifier

In [16]:
import urllib
from bs4 import BeautifulSoup

def web_scraper(url, fluff=0):
    """
    Grabs the text of the body for a given URL
    
    To-Do:
    - Remove '\n' tags
    
    - Extract number of LaTeX formulas
    - Extract number of images
    - Return all results in a dict
    """
    page = urllib.request.urlopen(url)
    soup = BeautifulSoup(page, 'html.parser')
    body = soup.find('body')
    return body.get_text()


web_scraper('https://jeffmacaluso.github.io/jekyll/update/2017/02/10/RestaurantRecommender.html')

'\n\n\nJeff Macaluso\n\n\n\n\n\n\n\n\n\n\n\n\nAbout\n\n\n\n\n\n\n\n\nRestaurant Recommender\n\n\n        \n        Feb 10, 2017\n      \n\n\n\nIntroduction\nIn this project, I extracted restaurant ratings and reviews from Foursquare and used distance (one of the main ideas behind recommender systems) to generate recommendations for restaurants in one city that have similar reviews to restaurants in another city.  This post is the abridged version, but check out my github post for all of the code if you are curious or want to use it.\nMotivation\nI grew up in Austin, Texas, and moved to Minneapolis, Minnesota a few years ago. My wife and I are people who love food, and loved the food culture in Austin. After our move, we wanted to find new restaurants to replace our favorites from back home.  However, most decently rated places in Minneapolis we went to just didn’t quite live up to our Austin expectations.  These restaurants usually came at the recommendations of locals, acquaintances, 

In [15]:
web_scraper('https://opendatascience.com/blog/curse-of-dimensionality-explained/')

'\n\n\n\n\n\n\n\nToggle navigation\n\n\n\n\n\nNews\nWebinar\nContent \n\nTalks\nBlog\n\n\nStart\nIntro\nJobs\n\n\n\nNews\nWebinar\nTalks\nBlogs\nStart\n\n\n\n\n\n\n\n\nSearch\t    \n\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n\n\n\n\n\n\n\n\nExact matches only        \n\n\n\n\n\n\n\nSearch in title        \n\n\n\n\n\n\n\nSearch in content        \n\n\n\n\n\n\n\nSearch in comments        \n\n\n\n\n\n\n\nSearch in excerpt        \n\n\nFilter by Custom Post Type\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nIntro\nJobs\n \n\n\n\n\n\n\n@media (max-width: 325px) {\n/*#header {top: 139px;}*/\n.home-widget-col-1 { margin-top:60px;}\n}\n\n\nvar date_new = "April 12, 2017 23:00";\nvar minutes =  4;\n \n\n\n\n\n\n\n\nCurse of Dimensionality Explained                                \n\n\n By Nikolay Manchev | 06/10/2016 \n\n\nTags: Machine Learning\n(function() {\nif (window.pluso)if (typeof window.pluso.start == "function") return;\nif